<a href="https://colab.research.google.com/github/sairamjadhav/sairamjadhav.github.io/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages and Tesseract OCR for image processing.
!pip install PyPDF2 pytesseract sentence-transformers faiss-cpu transformers
!apt-get install tesseract-ocr -qq

# Import necessary libraries.
import os
import io
from google.colab import files
import PyPDF2
import pytesseract
from PIL import Image
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import re


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
# Use Colab’s file upload widget.
uploaded = files.upload()

# Initialize an empty string to store all document text.
document_text = ""

# Loop through all uploaded files.
for filename in uploaded.keys():
    print(f"Processing {filename} ...")
    if filename.lower().endswith('.pdf'):
        # Process PDF file.
        with open(filename, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    document_text += text + "\n"
    elif filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        # Process image file with OCR.
        image = Image.open(filename)
        text = pytesseract.image_to_string(image)
        document_text += text + "\n"
    else:
        # Assume it’s a plain text file.
        with open(filename, "r", encoding="utf-8") as f:
            document_text += f.read() + "\n"

# Print a preview of the extracted text.
print("Extracted text preview:")
print(document_text[:500])


Saving IP for RAG.pdf to IP for RAG.pdf
Processing IP for RAG.pdf ...
Extracted text preview:
Key Points  
 
• TCS, or Tata Consultancy Services, is a major Indian IT company founded in 1968, 
headquartered in Mumbai, and operates globally with over 601,000 employees.  
 
• It seems likely that TCS offers services like IT consulting, digital transformation, and cloud 
computing, with a strong focus on AI and cybersecurity.  
 
• Research suggests TCS has a market cap of US$200 billion and generated US$29.1 billion in 
revenue in FY 2024, showing robust financial performance.  
 
• The ev


In [ ]:
def split_text(text, max_length=200):
    """
    Splits text into chunks of roughly max_length words.
    """
    # Split text into sentences using punctuation.
    sentences = re.split(r'(?<=[.?!])\s+', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        # If adding the sentence doesn’t exceed the max_length, add it.
        if len(current_chunk.split()) + len(sentence.split()) <= max_length:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Split the document text.
chunks = split_text(document_text)
print(f"Created {len(chunks)} text chunks.")


Created 8 text chunks.


In [ ]:
# Load a pre-trained Sentence Transformer model.
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all text chunks.
chunk_embeddings = embedder.encode(chunks, convert_to_tensor=False)
print("Embeddings created for each chunk.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings created for each chunk.


In [ ]:
# Determine embedding dimension from the first embedding.
embedding_dim = chunk_embeddings[0].shape[0]

# Create a FAISS index (using L2 distance).
index = faiss.IndexFlatL2(embedding_dim)

# Convert the list of embeddings to a NumPy array.
chunk_embeddings_np = np.array(chunk_embeddings).astype('float32')

# Add embeddings to the index.
index.add(chunk_embeddings_np)
print("FAISS index built with", index.ntotal, "embeddings.")


FAISS index built with 8 embeddings.


In [ ]:
# Choose a T5 model.
model_name = "t5-base"

# Load the tokenizer and model.
tokenizer = T5Tokenizer.from_pretrained(model_name)
gen_model = T5ForConditionalGeneration.from_pretrained(model_name)
print("T5 model and tokenizer loaded.")


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5 model and tokenizer loaded.


In [ ]:
def answer_question(question, top_k=3):
    # Compute the embedding for the question.
    q_embedding = embedder.encode([question])
    q_embedding_np = np.array(q_embedding).astype('float32')

    # Retrieve the indices of the top_k similar chunks.
    distances, indices = index.search(q_embedding_np, top_k)
    retrieved_chunks = [chunks[i] for i in indices[0]]

    # Combine retrieved chunks into a single context string.
    context = " ".join(retrieved_chunks)

    # Prepare input text for T5 in a prompt format.
    input_text = f"question: {question}  context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True)

    # Generate an answer (you can adjust max_length as needed).
    output_ids = gen_model.generate(input_ids, max_length=200)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return answer

# Test the function with an example question.
test_question = "Enter your test question here."
print("Answer:", answer_question(test_question))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Answer: Table


In [ ]:
# Interactive loop for asking questions.
print("You can now ask questions about your uploaded document. Type 'exit' to stop.")
while True:
    user_question = input("Enter a question: ")
    if user_question.lower().strip() == "exit":
        break
    print("Answer:", answer_question(user_question))


You can now ask questions about your uploaded document. Type 'exit' to stop.
Enter a question: When was TCS established?
Answer: 1968
Enter a question: What is revenue of TCS?
Answer: US$29.1 billion
Enter a question: How many employees does TCS have worldwide?
Answer: 601,000
Enter a question: Tell me about financial performance of TCS.
Answer: strong
Enter a question: Name one of the CEO's of TCS.
Answer: K. Krithivasan
Enter a question: exit
